In [1]:
#import all the required libraries
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import csv

# Task no.1 & 2

In [ ]:
#lets first connect to the webdriver
driver=webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(6)

url = "https://www.amazon.in/"
driver.get(url)
time.sleep(5)

# entering the data into Search bar
search_field=driver.find_element_by_id("twotabsearchtextbox") #product search bar
search_field.send_keys("watches")

# clicking the search button
search_button=driver.find_element_by_xpath("//*[@id='nav-search-submit-button']")
search_button.click()
time.sleep(4)

# creating empty lists for scraping data
Brand_name=[]
Price=[]
Product_URL=[]
Rating=[]
Numb_of_rating=[]
time.sleep(3)

urls=[]
#scrapping the required details
start=0
end=3
for page in range(start,end):
    brands=driver.find_elements_by_class_name('s-line-clamp-1')
    for i in brands:
        Brand_name.append(i.text)
#scrapping Price
    prices=driver.find_elements_by_xpath("//span[@class='a-price-whole']")
    for i in prices:
        Price.append(i.text)
    nxt_button=driver.find_elements_by_xpath("//li[@class='a-last']")
    try:
        driver.get(nxt_button[1].get_attribute('href'))
    except:
        driver.get(nxt_button[0].get_attribute('href'))

time.sleep(2)
for page in range(0,3):
    product_url = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
    for i in product_url:
        Product_URL.append(i.get_attribute('href'))
    nxt_button=driver.find_elements_by_xpath("//li[@class='a-last']")
    try:
        driver.get(nxt_button[1].get_attribute('href'))
    except:
        driver.get(nxt_button[0].get_attribute('href'))
        
# scrapping rating of the product
urls=driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
UR=[]
for i in urls[0:7]:
    UR.append(i.get_attribute('href'))
for url in UR:
    driver.get(url)
    try:                                                                      
        rating=driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']//span")
        Rating.append(rating.text)
        
    except NoSuchElementException:
        Rating.append("NO rating")
        
#number of rating
urls=driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
UR=[]
for i in urls[0:7]:
    UR.append(i.get_attribute('href'))
for url in UR:
    driver.get(url)
    try:                                                                      
        ratings=driver.find_element_by_xpath("//span[@class='a-link-normal']")
        Numb_of_rating.append(rating.text)
        
    except NoSuchElementException:
        Numb_of_rating.append("-")
        
df=pd.DataFrame({"Brand":Brand_name,"Rating":Rating,"Number_of_rating":Numb_of_rating,
                 "Price":Price,"url":Product_URL})
#saving the data into csv file
df.columns = ['Brand', 'Rating', 'Number_of_rating', 'Price', 'Product_URL']
df.to_csv('Watch_details.csv', index=False)

# Task no.3

Write a python program to access the search bar and search button on images.google.com and 
scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [13]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os
import time
import urllib.request
from selenium.webdriver.common.keys import Keys

In [8]:
driver=webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")
driver.get("https://www.google.com/")
search = driver.find_element_by_name('q')
search.send_keys('fruits',Keys.ENTER)

#entered the search and need to switch to images tab on the browse page to search for images.
elem = driver.find_element_by_link_text('Images')
elem.get_attribute('href')
elem.click()

#
value = 0
for i in range(20): 
    driver.execute_script('scrollBy("+ str(value) +",+100);')
    value += 100
time.sleep(4)
#
elements = driver.find_elements_by_xpath('//img[contains(@class,"rg_i")]')
#
for i in elements:
    src = i.get_attribute('src')
    try:
        if src != None:
            src  = str(src)
            count=1
            urllib.request.urlretrieve(src, os.path.join('fruits','fruits'+str(count)+'.jpg'))
        else:
            raise TypeError
    except TypeError:
        pass
      

# Task no.5

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on 
google maps.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
import time
from openpyxl import load_workbook
from random import randint

wb_name = "Book1.xlsx" #file name


wb = load_workbook(wb_name, data_only = True)
ws = wb['sheet_name'']
address_list =[]
link_col = A1  #column number

coord_prospects = pd.DataFrame() 

for row in ws.iter_rows(min_row = x , max_row = x, min_col = link_col, max_col=link_col):
    if str(row[0].value) != "None":
        address_list.append(row[0].value)

driver=webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")
#driver.minimize_window() #this is optional if the opening google chrome window gets annoying
driver.get('https://www.mapdevelopers.com/geocode_tool.php')
add = driver.find_element_by_class_name('form-control')

for t,a in enumerate(address_list):
    print ("Geocoding...",t+1,"/",len(address_list),str(round(t/len(address_list)*100,2)),"%"," : ", a)
    add.clear()
    add.send_keys(a)
    try:
        search1 = driver.find_element_by_xpath('//*[@id="search-form"]/div[1]/span[2]').click()
        time.sleep(3)
        search2 = driver.find_element_by_xpath('//*[@id="search-form"]/div[1]/span[2]').click()
        time.sleep(3)
    except ElementClickInterceptedException:
        time.sleep(2)
        search = driver.find_element_by_xpath('//*[@id="search-form"]/div[1]/span[2]').click()
    lat=driver.find_element_by_id('display_lat')
    lng=driver.find_element_by_id('display_lng')
    street=driver.find_element_by_id('display_address')
    city=driver.find_element_by_id('display_city')
    postcode=driver.find_element_by_id('display_zip')
    state=driver.find_element_by_id('display_state')
    county=driver.find_element_by_id('display_county')
    country=driver.find_element_by_id('display_country')
    latlng = pd.DataFrame({'Latitude':pd.Series(lat.text),
                            'Longitude':pd.Series(lng.text),
                            'Street':pd.Series(street.text),
                            'City':pd.Series(city.text),
                            'Postcode':pd.Series(postcode.text),
                            'State':pd.Series(state.text),
                            'County':pd.Series(county.text),
                            'Country':pd.Series(country.text)})
    coord_prospects = coord_prospects.append(latlng, ignore_index=True)
    print(coord_prospects.tail(1))
    print("   ")

coord_prospects.to_excel('Book1.xlsx') #name of output excel file

driver.close()

# Task no.7

Write a program to scrap all the available details of top 10 gaming laptops from digit.in

In [ ]:
#lets first connect to the webdriver
driver=webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(6)

url = "https://www.digit.in/"
driver.get(url)
time.sleep(5)

#clicking on search feild
search_field=driver.find_element_by_class_name("search").click()

#Entering details on search keys
search=driver.find_element_by_id("globalPageSearchText")
search.send_keys("gamming laptops", Keys.ENTER)

#creating the empty list
product_name=[]
description=[]
price=[]
review=[]
page_urls = []

# scrape next pages urls
nxt_page = driver.find_elements_by_xpath("//li[@class='pagination-number']/a")
for i in nxt_page:
    page_urls.append(i.get_attribute('href'))
for url in page_urls[:3]:
    driver.get(url)
    Names=driver.find_elements_by_xpath("//div[@class='searchProduct-desc'])
    for i in Names:
        product_name.append(i.text)
    
    desc=driver.find_elements_by_xpath("//div[@class='MoreLink-briefdescription']")#for scrapping description
    for i in desc:
        description.append(i.text)
    
    rs=driver.find_elements_by_xpath("//div[@class='product-price']")  #for scrapping prices
    for i in rs:
        price.append(i.text)
                                        
    rv=driver.find_elements_by_xpath("//div[@class='UserDate greentext']")#for scrapping review
    for i in rv:
        review.append(i.text)
                                        
df=pd.DataFrame({'Brand': product_name[:10],'description': description[:10],'Price': price[:10],'Review': review[:10]})
df        

# Task no.4

Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on 
www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, 
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display 
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. 
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.

In [ ]:
# Importing Libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

# creating empty list
brand_name = []    
RAM=[]
Display=[]
Camera=[]
Battery=[]
Processor=[]
Accessories=[]
Price=[]
rating=[]

pages= list(range(1,2))
for page in pages:
    req=requests.get("https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={}".format(page)).text
    #content of the page
    soup=BeautifulSoup(req,'html.parser')
    
    brand=soup.find_all('div',class_='_4rR01T')
    for i in range(len(brand)):
        brand_name.append(brand[i].text)
        len(brand_name)
        
    commonclass=soup.find_all('li',class_='rgWa7D')
    #create empty list for the features
    for i in range(0,len(commonclass)):
        p=commonclass[i].text
        if("RAM" in p):
            RAM.append(p)
        elif("Display" in p):
            Display.append(p)
        elif("Camera" in p):
            Camera.append(p)
        elif("Battery" in p):
            Battery.append(p)
        elif("Processor" in p):
            Processor.append(p)
        elif("Accessories" in p):
            Accessories.append(p)
        
    Price=soup.find_all('div',class_='_30jeq3 _1_WHN1')
    for i in range(len(Price)):
        Price.append(Price[i].text)
        len(Price) 
  
    rating=soup.find_all('div',class_='_3LWZlK')
    for i in range(len(rating)):
        rating.append(rating[i].text)
        len(rating)
        
#keeping into dataframe       
df= {'brand_name':brand_name[40],'RAM':RAM[40],'Display':Display[slice(40)],'Camera':Camera[slice(40)],'Battery':Battery[slice(40)],'Processor':Processor[slice(40)],'Accessories':Accessories[slice(40)],'Price':Price[40],'rating':rating[40]}
dataset=pd.DataFrame(data = df)

#storing in csv file
dataset.to_csv('smartphone.csv')


# Task no.6

Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in.